# Lab 10 - Sentiment Analysis 
**COMP130 - Introduction to Computing**  
**Dickinson College**  

### Names:

### Automated Sentiment Analysis

Sentiment analysis is the process of reading some text and determining the author's feelings (positive or negative) toward the subject on which they are writing. For example consider the following two movie reviews: 

- It almost feels as if the movie is more interested in entertaining itself than in amusing us.
- A positively thrilling combination of ethnography and all the intrigue, betrayal, deceit and murder of a Shakespearean tragedy or a juicy soap opera.

It is pretty clear to us as human readers that the first review expresses a negative sentiment while the second one expresses a positive sentiment.  The task then for *automated sentiment analysis* is for the computer to take a piece of text and determine if is is expressing a positive or negative sentiment. 

While this might sound like a daunting task, there is a relatively clever way to do it if we have a large data set with words that have a known sentiment somehow already attached to them. As it turns out, there are lots and lots of such data sets. In fact, every review site on the the Internet has one:
- "5 stars: Loved it!"
- "0 stars: Absolutely pathetic!" 

The basic idea then is that these reviews can be used to associate a positive sentiment ("5 stars") the words "Loved" and "it" and a negative sentiment ("0 stars") to the words "Absolutely" and "pathetic". We'll get more into exactly how it will work below.

### The Data Set:

The data set that we will use to assign sentiments is based on movie reviews and is from real movie reviews posted at [Rotten Tomatoes](https://www.rottentomatoes.com/).  This data set was originally produced for a [Kaggle machine learning competition](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/overview) and has since been adapted for a similar assignment by Eric D. Manley and Timothy M. Urness at Drake University.  It is their data set that we are using.

__Q1:__ The data set has been broken into two pieces. One we will use as *training data* to assign sentiments to words in our sentiment analysis program.  The other we will use as *validation data* to see how good our program does at figuring out the sentiment of a review.  The training data set is stored in a file name `training.txt` and the validation data is stored in a file named `validation.txt`.  Both of the files have the same format but contain different reviews.  Open the `training.txt` file in a text editor and copy the first 5 lines of the file into the cell below.  

__Q2:__ Each line has two parts. The first part is a single number indicating the rating given by the reviewer and the second part is the text of the review.  What is the lowest rating in the `training.txt` file?  What is the highest rating in the file?

__Q3:__ Which rating represents the most negative sentiment?  Give the text of the review for one such rating.

__Q4:__ Which rating represents the most positive sentiment? Give the text of the review for one such rating.

### Reading the Data Set

__Q5:__ Write a statement that opens the data file for the training data.

__Q6:__ Write some statements that read and print the first line of the training data file.

__Q7:__ What happens if you run your statement from __Q6__ a few more times?

__Q8:__ When a program is finished reading data from a file it should also close the file.  This lets the operating system know it is no longer being used. The statement `datafile.close()` would close a file that was opened and assigned to the variable `datafile`.  Write a statement below that closes the file you opened in `Q5`.

__Q9:__ What happens now if you run your statement from __Q6__?

__Q10:__ Consider the following program.  It opens the `training.txt` file, reads every line in the file, counting them as it goes.  When it has read all of the lines in the file it closes the file and then prints out how many lines there are.  Note: This is slightly different than the `for in` loop the book uses.  Both approaches are valid.

In [ ]:
train=open('training.txt')

count = 0

line = train.readline()
while line:
    count=count+1
    line = train.readline()
    
train.close()
print('training.txt has ' + str(count) + ' lines.')

How many lines are in the training data set?  How many lines are in the validation training set?

__Q11:__ Based on the behavior of the program in __Q10__ what value does the `readline` function return when all of the lines in the file have been read? I.e. What is the value of `line` that causes the `while` loop to terminate?

### Pre-Processing the Lines

The lines in the data file's have two parts, the *sentiment rating* of the review and the *review text*. You will need those pieces to be separated from each other in order to use them for sentiment analysis. In addition the reviews have a mixture of uppercase and lowercase letters and include punctuation and sometimes extra spaces at the beginning and end of the files. 

Often in applications where large data sets are being used some *pre-processing* of the data to *clean* it up. Cleaning up the data makes writing the actual program easier because it does not have to deal with any special cases (e.g. punctuation, upper vs lower case, etc).  In this section you'll write functions that will *pre-process* a line from the data set by splitting the *sentiment rating* from the *review text* and clean up the *review text*.

__Q12:__ Write a function named `get_sentiment_rating` that takes a string containing a sentiment rating and the associated review text (i.e. like a line from the data set) and returns an `int` value that is the sentiment rating.  Use the automated tests below to check that your function is working correctly.

In [ ]:
assert get_sentiment_rating('4 I laughed, I cried, it was better than cats.')==4, 'sentiment should be 4'
assert get_sentiment_rating('0 Two thumbs down.')==0, 'sentiment should be 0'
print("Success!")

__Q13:__ Write a function named `get_review_text` that takes a string containing a sentiment rating and the associated review text (i.e. like a line from the data set) and returns a `String` containing the review text.

__Q14:__ Write a few automated tests that check that your `get_review_text` function works correctly.

__Q15:__ Write a function named `clean_text` that takes in a `String` containing some text and returns a *cleaned up* version by making it all lower case, removing all spaces from the beginning or end and removing all characters that is not a space or a alphabetic character. Hint: Make use of some of the `String` functions to help you.  Use the automated tests below to check that your function is working correctly.

In [ ]:
assert clean_text('I lauGHed I CrieD')=='i laughed i cried', 'review text is not all lowercase'
assert clean_text('  Two ThumBS DowN  ')=='two thumbs down', 'spaces at start and end not removed'
assert clean_text('  This "1" wasn`t BAD!!!!  ')=='this  wasnt bad', 'non alphabetic characters not removed'
print('Success!')

__Q16:__ We are not quite ready to do sentiment analysis yet, but when working with data sets sometimes it is useful to write a small program to be sure you can do *something* with the data before trying to do exactly what you want. This is really just the idea of *incremental development* again.  To practice this and be sure you re ready to go on... Write a program that prints out the average sentiment rating in the `training.txt` file.  Hints: use the program in __Q10__ as a model and make use of the `get_sentiment_rating` function you wrote in __Q12__.  If your program works you should find an average rating of (`2.0626557493770026`).

### A Simple Sentiment Analysis Algorithm 

The introduction gave a hint as to how we might compute the sentiment of a word.


We will use the sentiments attached to a review as a proxy for the sentiment attached to the words they contain. For example, if a given word (e.g. "pathetic") shows up in 6 reviews with the ratings 1, 2, 0, 0, 1, and 2 then we might assign "pathetic" a *sentiment score* of 1+2+0+0+1+1=5/6=0.83. The sentiment of some new text could then be computed as the average of the sentiment scores of the words it contains.  You may already be thinking that it would be pretty easy to fool this algorithm, and you'd be right.  However, we'll see that it really does surprisingly well.

__Q17:__ Write a function named `word_sentiment` that takes a `String` containing a single word as an argument and returns the *sentiment rating* of that word based on all of the reviews in the `training.txt` file.  Hint: Use the program you wrote in __Q16__ as a starting point and use the `get_sentiment_rating`, `get_review_text` and `clean_text` functions you wrote earlier to do some of the work for you. Use the automated tests below to check that your function is working correctly.

In [ ]:
epsilon = 0.0001
assert abs(word_sentiment('Fabulous')-2.75) < epsilon, 'Incorrect sentiment for fabulous.'
assert abs(word_sentiment('boring')-1.1428571428571428) < epsilon, 'Incorrect sentiment for boring'
assert abs(word_sentiment('love')-2.574660633484163) < epsilon, 'Incorrect sentiment for love'
assert abs(word_sentiment('hate')-1.7954545454545454) < epsilon, 'Incorrect sentiment for hate'
assert abs(word_sentiment('the')-2.038604742308446) < epsilon, 'Incorrect sentiment for the'
print('Success!')

__Q18:__ The expected sentiment ratings of the words in the above tests seem to make pretty good sense. `Fabulous` is pretty positive while `boring` is pretty negative.  Compute the sentiment rating of some other words and list them below. How did the program do?  Do the sentiments of the words you tried seem about right?

__Q19:__ Does your `word_sentiment` function work if the word passed to it does not appear in any review in the training data? What value could your `word_sentiment` function return in this situation so as not to adversely affect the results? 

__Q20:__ Modify your `word_sentiment` function above and then write an automated test below that checks that it now handles words that do not appear in any training data in a reasonable way.

### The Sentiment of Multiple Words

__Q21:__ The next step will be to compute the sentiment of a line containing multiple words. Before doing that consider the program below which will ultimately help you.

In [ ]:
line = "this is a test"
for word in line.split():
    print(word)

What does the above program do?  How might this help with computing the sentiment of a line containing multiple words?

__Q22:__ Write a function named `text_sentiment` that takes a `String` with multiple words in it as an argument and computes the sentiment of the text as the average sentiment rating of all of the words. Hint: Use your `word_sentiment` function from __Q17__ with the loop in __Q21__.

__Q23:__ Write a program that prompts the user to enter some text and then responds to the user with an appropriate message (be creative) depending upon whether the input had a positive sentiment or a negative sentiment. Add more responses for more positive or negative or neutral sentiments if you like. 

__Q24:__ Try out your program in __Q23__. What are some phrases you tried where the program got the sentiment right? What is one where it got it wrong? Why do you think it got that one wrong?

### How Good Is It?

A reasonable question to ask now is how good is this algorithm really? That's where the *validation data* comes in. Each review in the `validation.txt` file has a review and a human decided sentiment rating.  So we can submit each review in the `validation.txt` file to the `text_sentiment` function and then compare the result it gives to the human decided sentiment rating. Because the data in the `validation.txt` file is not used in computing the sentiment rating, this is like showing the algorithm a new rating text and asking it what it thinks the sentiment should be. But by having the human rating, we can check out good the algorithm is doing.

__Q25:__ Write statements that read the first review from the `validation.txt` file and splits it into its sentiment rating and review text.

__Q26:__ Write statements that uses your `text_sentiment` function to compute the sentiment rating of the review text from __Q25__, then displays the human sentiment rating, the algorithms sentiment rating and the error between the two (i.e. the absolute value of their difference).

__Q28:__ Write a program that computes the average error for all of the reviews in the `validate.txt` file.  Have your program print out the error for each review as it goes and then print the average error at the end.  Note: It will take a little while for this program to run.

__Q29:__ For a relatively simple algorithm the results from __Q28__ might be surprisingly good. But based on modern experiences we might expect that the computer could do much better.  What are some factors that you think  limited the accuracy of the results? Or what are some ways you think the algorithm could do better?

### A Note on Efficiency

The program from __Q28__ took quite a long time to run. If you think about it this may not be very surprising. The program actually reads trough the entire `training.txt` file for every word in the `validation.txt` file. If you open the `validation.txt` file you'll see there are a lot of words in there.  So that means that the `training.txt` file is being read a lot of times. Add to that the fact that reading through a file is a relatively slow process by comparison to most operations the computer performs and the result is a slow program.  There are a number of more efficient ways to handle the data in the `training.txt` file that could make this program run much more quickly.  We'll be learning about a few of those *lists* and *dictionaries* later in the semester.

### Optional Extra Challenges

1. Some words like "a", "and", "the", etc... appear in a lot of reviews, but do not really contribute to the sentiment of the review. Create a text file named `skip.txt` that holds a list of words that should be skipped when processing the reviews. Then modify your program so that the words `skip.txt` are not included in the sentiment analysis.
1. Learn how to create a [Histogram with the Matplotlib](https://pythonspot.com/matplotlib-histogram/) module and generate a histogram of the error values for the validation data set.
1. Think about and implement one or more improvements to this algorithm that you think will reduce the error in the sentiment ratings on the validation set.  Describe your improvements and indicate how much they reduced the error. 

---
__Acknowledgment:__ This lab has been based on the ideas and dataset created by Eric D. Manley and Timothy M. Urness
Drake University for their assignment [Movie Review Sentiment Analysis](http://nifty.stanford.edu/2016/manley-urness-movie-review-sentiment/) as contained in the [Nifty Assignments](http://nifty.stanford.edu/) repository.
